In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
# <---- You will also likely want to use the "folder" icon to add some files
#       for the agent to look at
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
tools = {}
tools_by_tag = {}

def to_openai_tools(tools_metadata: List[dict]):
  openai_tools = [
      {
          "type": "function",
          "function": {
              "name": t["tool_name"],
              # Include up to 1024 characters of the description
              "description": t.get('description', "")[:1024],
              "parameters": t.get('parameters', {})
          }
      } for t in tools_metadata
  ]
  return openai_tools

def get_tool_metadata(fuct, tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
  """
  Extracts metadata for a function to use in tool registration.

  Parameters:
  - func (function): The function to extract metadata from.
  - tool_name (str, optional): The name of the tool. Defaults to the function name.
  - description (str, optional): Description of the tool. Defaults to the function name.
  - parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
  - terminal (bool, optional): Whether the tool is terminal. Defaults to false.
  - tags (List[str], optional): List of tags to associate with the tool.

  Returns:
  - dict: A dictionary containing the tool metadata, including description, args, schema, and the function.
  """
  # Default tool_name to the function name if not provided
  tool_name = tool_name or func.__name__

  # Default description to the function's docstring if not provided
  description = description or (func.__doc__.strip() if func.__doc__ else "No description provided.")

  # Discover the function's signature and type hints if no args_override is provided
  if parameters_override is None:
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    # Build the arguments schema dynamically
    args_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }

    for param_name, param in signature.parameters.items():

        if param_name in ["action_context", "action_agent"]:
          continue # Skip these parameters

        def get_json_type(param_type):
          if param_type == str:
            return "string"
          elif param_type == int:
            return "integer"
          elif param_type == float:
            return "number"
          elif param_type == bool:
            return "boolean"
          elif param_type == list:
            return "array"
          elif param_type == dict:
            return "object"
          else:
            return "string"

        # Add parameter details
        param_type = type_hints.get(param_name, str) # Default to string if type is not annotated
        param_schema = {"type": get_json_type(param_type)} # Convert Python types to JSON Schema types

        args_schema["properties"][param_name] = param_schema

        # Add to required if not defaulted
        if param.default is not param.empty:
            args_schema["required"].append(param_name)
    else:
      args_schema = parameters_override

    # Return the metadata as a dictionary
    return {
        "tool_name": tool_name,
        "description": description,
        "parameters": args_schema,
        "function": func,
        "terminal": terminal,
        "tags": tags or []
    }

  def register_tool(tool_name=None, description=None, parameters_override=None, terminal=False, tags=None):
    """
    A decorator to dynamically register a function in the tools dictionary with its parameters, schema, and docstring.

    Parameters:
        tool_name (str, optional): The name of the tool to register. Defaults to the function name.
        description (str, optional): Override for the tool's description. Defaults to the function's docstring.
        parameters_override (dict, optional): Override for the argument schema. Defaults to dynamically inferred schema.
        terminal (bool, optional): Whether the tool is terminal. Defaults to False.
        tags (List[str], optional): List of tags to associate with the tool.

    Returns:
        function: The wrapped function.
    """

    def decorator(func):
      # Use the reusable function to extract metadata
      metadata = get_tool_metadata(
          func=func,
          tool_name=tool_name,
          description=description,
          parameters_override=parameters_override,
          terminal=terminal,
          tags=tags
      )

      # Register the tool in the global dictionary
      tools[metadata["tool_name"]] = {
          "description": metadata["description"],
            "parameters": metadata["parameters"],
            "function": metadata["function"],
            "terminal": metadata["terminal"],
            "tags": metadata["tags"] or []
      }

      for tag in metadata["tags"]:
        if tag not in tools_by_tag:
          tools_by_tag[tag] = []
        tools_by_tag[tag].append(metadata["tool_name"])

      return func
    return decorator

In [ ]:
@register_tool()
def prompt_llm_for_json(action_context: ActionContext, schema: dict, prompt: str):
    """
    Have the LLM generate JSON in response to a prompt. Always use this tool when you need structured data out of the LLM.
    This function takes a JSON schema that specifies the structure of the expected JSON response.

    Args:
        schema: JSON schema defining the expected structure
        prompt: The prompt to send to the LLM

    Returns:
        A dictionary matching the provided schema with extracted information
    """
    generate_response = action_context.get("llm")

    # Try up to 3 times to get valid JSON
    for i in range(3):
        try:
            # Send prompt with schema instruction and get response
            response = generate_response(Prompt(messages=[
                {"role": "system",
                 "content": f"You MUST produce output that adheres to the following JSON schema:\n\n{json.dumps(schema, indent=4)}. Output your JSON in a ```json markdown block."},
                {"role": "user", "content": prompt}
            ]))

            # Check if the response has json inside of a markdown code block
            if "```json" in response:
                # Search from the front and then the back
                start = response.find("```json")
                end = response.rfind("```")
                response = response[start+7:end].strip()

            # Parse and validate the JSON response
            return json.loads(response)

        except Exception as e:
            if i == 2:  # On last try, raise the error
                raise e
            print(f"Error generating response: {e}")
            print("Retrying...")